### **ENVIRONMENT SETUP**

In [1]:
# ! rm -r data*
# ! wget http://argumentation.bplaced.net/arguana-data/dagstuhl-15512-argquality-corpus-v2.zip
# ! unzip dagstuhl-15512-argquality-corpus-v2.zip
# ! rm *.zip
# ! rm -r __MACOSX
# ! mv dagstuhl-15512-argquality-corpus-v2 data
# ! pip install -q pyLDAvis nltk seaborn scikit-learn matplotlib wordcloud

### **IMPORT LIBRARIES**

In [2]:
# Set random seed

import random
random.seed(14071)

In [2]:
import re
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer # Bag of Words
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF

from sklearn.preprocessing import LabelEncoder # Label Encoding
from sklearn.preprocessing import OneHotEncoder # One Hot Encoding

from sklearn.model_selection import train_test_split # Train Test Split

from sklearn.linear_model import LogisticRegression # LR Model
from sklearn.tree import DecisionTreeClassifier # DT Model
from sklearn.ensemble import RandomForestClassifier # RF Model

import tensorflow.keras as keras # Keras bindings
import tensorflow as tf # Tensorflow bindings

from sklearn.metrics import classification_report # Classification Report

from sklearn.model_selection import GridSearchCV # Grid Search

[nltk_data] Downloading package stopwords to /home/sri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sri/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2022-09-19 16:12:42.027942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-19 16:12:43.331069: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot ope

### **IMPORT DATA**

In [4]:
df = pd.read_csv("data/dagstuhl-15512-argquality-corpus-annotated.csv", sep='\t', encoding_errors="ignore")

In [5]:
df

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance
0,1,y,1 (Low),1 (Low),1 (Low),1 (Low),2 (Average),1 (Low),1 (Low),1 (Low),...,1 (Low),1 (Low),1 (Low),1 (Low),1 (Low),1 (Low),"it is true that bottled water is a waste, but ...",arg219250,ban-plastic-water-bottles,no-bad-for-the-economy
1,2,y,1 (Low),3 (High),2 (Average),2 (Average),3 (High),1 (Low),1 (Low),3 (High),...,1 (Low),2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),"it is true that bottled water is a waste, but ...",arg219250,ban-plastic-water-bottles,no-bad-for-the-economy
2,3,y,2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),2 (Average),1 (Low),2 (Average),"it is true that bottled water is a waste, but ...",arg219250,ban-plastic-water-bottles,no-bad-for-the-economy
3,1,y,2 (Average),3 (High),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),3 (High),2 (Average),2 (Average),Most Americans on average recycle 86-88% of th...,arg219293,ban-plastic-water-bottles,no-bad-for-the-economy
4,2,y,1 (Low),2 (Average),1 (Low),2 (Average),2 (Average),1 (Low),1 (Low),2 (Average),...,1 (Low),1 (Low),2 (Average),2 (Average),2 (Average),1 (Low),Most Americans on average recycle 86-88% of th...,arg219293,ban-plastic-water-bottles,no-bad-for-the-economy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2,y,2 (Average),2 (Average),3 (High),2 (Average),2 (Average),1 (Low),1 (Low),3 (High),...,1 (Low),2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),Raffles neglected Singapore when he went aroun...,arg168822,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course
956,3,y,2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),2 (Average),2 (Average),2 (Average),Raffles neglected Singapore when he went aroun...,arg168822,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course
957,1,y,2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),2 (Average),2 (Average),2 (Average),"Raffles doesn't care about the citizens, doesn...",arg168834,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course
958,2,y,2 (Average),2 (Average),3 (High),2 (Average),3 (High),1 (Low),2 (Average),2 (Average),...,1 (Low),2 (Average),2 (Average),2 (Average),3 (High),1 (Low),"Raffles doesn't care about the citizens, doesn...",arg168834,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course


In [6]:
print(f"Number of annotations = {len(df['argument'])}")
print(f"Number of unique arguements = {len(np.unique(df['argument']))}") # Each argument was scored by 3 annotators
print(f"Number of unique issue = {len(np.unique(df['issue']))}")  # There are a total of 16 issues
print(f"Number of unique stance = {len(np.unique(df['stance']))}") # Each issue has on an avg 2 stance (positive and negative)

Number of annotations = 960
Number of unique arguements = 320
Number of unique issue = 16
Number of unique stance = 28


### **DATA CLEANING**

#### Remove statements are that are tagged as NOT argumentative

In [7]:
df[df["argumentative"] == "n"] # Statements that are tagged as NOT argumentative

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance
25,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,We will be able to ban water bottles until we ...,arg219242,ban-plastic-water-bottles,no-bad-for-the-economy
32,3,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The high price of bottled water is not the wat...,arg219232,ban-plastic-water-bottles,yes-emergencies-only
37,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,A drop of water is worth more than a sack of g...,arg219210,ban-plastic-water-bottles,yes-emergencies-only
51,1,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yeah I have a bottle of water next to me its n...,arg219292,ban-plastic-water-bottles,yes-emergencies-only
52,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yeah I have a bottle of water next to me its n...,arg219292,ban-plastic-water-bottles,yes-emergencies-only
53,3,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yeah I have a bottle of water next to me its n...,arg219292,ban-plastic-water-bottles,yes-emergencies-only
97,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,This is just wrong we should not insult who we...,arg236317,christianity-or-atheism,christianity
104,3,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,I have a personal relationship with Christ. I ...,arg317490,christianity-or-atheism,christianity
105,1,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,God helps those who help themselves! So i will...,arg234318,christianity-or-atheism,christianity
106,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,God helps those who help themselves! So i will...,arg234318,christianity-or-atheism,christianity


In [8]:
statements = df[df["argumentative"] == "n"]["argument"].to_numpy() # Extract the statements
statements = np.unique(statements) # Extract the unique statements

for ele in statements: # Remove all occurrences of NOT argumentative statements
    df.drop(df[df['argument'] == ele].index, axis=0, inplace=True)

In [9]:
df[df["argumentative"] == "n"]

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance


In [10]:
print(f"Number of annotations = {len(df['argument'])}")
print(f"Number of unique arguements = {len(np.unique(df['argument']))}") # Each argument was scored by 3 annotators
print(f"Number of unique issue = {len(np.unique(df['issue']))}")  # There are a total of 16 issues
print(f"Number of unique stance = {len(np.unique(df['stance']))}") # Each issue has on an avg 2 stance (positive and negative)

Number of annotations = 912
Number of unique arguements = 304
Number of unique issue = 16
Number of unique stance = 28


#### Combine all Annotators' scores into a single score

In [11]:
argument = np.unique(df["argument"])

In [12]:
attributes = ["annotator", "overall quality", "cogency", "effectiveness", "reasonableness", "argument", "#id"]

cleaned_df = []

for arg in argument:

    new_df = df[df["argument"] == arg][attributes]
    flag = 0
    new_dict = {
        "#id": new_df["#id"].iloc[0],
        "argument": new_df["argument"].iloc[0],
    }

    for ele in ["overall quality", "cogency", "effectiveness", "reasonableness"]:
        if len(pd.value_counts(new_df[ele])) == 3:
            flag = 1
            break
        new_dict[ele] = pd.value_counts(new_df[ele]).index[0]
        
    if flag == 1:
        continue
    cleaned_df.append(new_dict)

cleaned_df = pd.DataFrame(cleaned_df)

In [13]:
df[df["#id"] == "28068"]

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance
144,1,y,2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),1 (Low),1 (Low),...,1 (Low),1 (Low),3 (High),1 (Low),2 (Average),1 (Low),"""Debates are based on convincing evidence. The...",28068,evolution-vs-creation,creation
145,2,y,1 (Low),1 (Low),3 (High),2 (Average),2 (Average),1 (Low),1 (Low),1 (Low),...,1 (Low),1 (Low),2 (Average),1 (Low),2 (Average),1 (Low),"""Debates are based on convincing evidence. The...",28068,evolution-vs-creation,creation
146,3,y,2 (Average),2 (Average),2 (Average),2 (Average),3 (High),2 (Average),1 (Low),2 (Average),...,1 (Low),1 (Low),2 (Average),1 (Low),2 (Average),2 (Average),"""Debates are based on convincing evidence. The...",28068,evolution-vs-creation,creation


In [14]:
cleaned_df

,#id,argument,overall quality,cogency,effectiveness,reasonableness
0,28068,"""Debates are based on convincing evidence. The...",2 (Average),1 (Low),1 (Low),1 (Low)
1,13270,"""If a women is raped"" is a good argument. Howe...",1 (Low),1 (Low),1 (Low),1 (Low)
2,13275,"""The government has no place to tell a woman w...",1 (Low),1 (Low),1 (Low),1 (Low)
3,12365,(I am writing this through Firefox) Emotions a...,2 (Average),1 (Low),1 (Low),2 (Average)
4,arg561672,1. It makes everyone equal - if children can w...,1 (Low),2 (Average),1 (Low),2 (Average)
...,...,...,...,...,...,...
256,arg334959,"yea, because even though there are many other ...",2 (Average),2 (Average),1 (Low),2 (Average)
257,arg335089,yes because if they fear getting hit than they...,1 (Low),1 (Low),1 (Low),1 (Low)
258,arg203922,"yes, i believe it's nice to have a school unif...",2 (Average),2 (Average),1 (Low),2 (Average)
259,arg596217,"yes,India has potential to lead the world.So, ...",1 (Low),1 (Low),1 (Low),1 (Low)


In [15]:
print(f"Number of arguements = {len(cleaned_df['argument'])}")

Number of arguements = 261


### **DATA PREPROCESSING**

In [16]:
text = cleaned_df["argument"]

In [17]:
stop_words = set(stopwords.words("english"))
english_stopwords = stopwords.words("english")
english_stemmer = SnowballStemmer("english")

In [18]:
def clean_text(text):
    text = text.replace('</br>', '') # Remove </br>
    text = re.sub(r'[^\w]', ' ', text) # Remove symbols
    text = re.sub(r'[ ]{2,}', ' ', text) # Remove extra spaces
    text = re.sub(r'[ \t]+$', '', text) # Remove trailing white spaces
    tokens = []
    for token in text.split():
        if token not in stop_words:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)
    #return token

In [19]:
cleaned_text = [clean_text(text) for text in text]
text = cleaned_text

### **VECTORIZE THE TEXT DATA**

In [20]:
# Using Bag of Words (BoW)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

print(f"Shape of Vector = {X.shape}")

Shape of Vector = (261, 2001)


In [21]:
# # Using TF-IDF

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(text)
# X = X.toarray()

# print(f"Shape of Vector = {X.shape}")

### **PREDICTING OVERALL QUALITY FROM TEXT**

In [22]:
y = cleaned_df["overall quality"].to_numpy()
y = y.reshape(-1, 1)

#### Label Encoding

In [23]:
# Label Encoding

encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)

print(f"Size of Labels = {enc_y.shape}")

Size of Labels = (261,)


/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
# # One Hot Encoding

# encoder = OneHotEncoder()
# enc_y = encoder.fit_transform(y)
# enc_y = enc_y.toarray()

# print(f"Size of Labels = {enc_y.shape}")
# print(f"Label Sample: {enc_y[0]}")

#### Train Test Split

In [25]:
X_train, X_test, y_train, y_test= train_test_split(X, enc_y, test_size=0.2, random_state=110)

print(f"Shape of Training Data: {X_train.shape}")
print(f"Shape of Training Labels: {y_train.shape}")
print(f"Shape of Testing Data: {X_test.shape}")
print(f"Shape of Testing Labels: {y_test.shape}")

Shape of Training Data: (208, 2001)
Shape of Training Labels: (208,)
Shape of Testing Data: (53, 2001)
Shape of Testing Labels: (53,)


In [3]:
pd.value_counts(cleaned_df["overall quality"])

NameError: name 'cleaned_df' is not defined

#### Training Logistic Regression Model

In [26]:
model = LogisticRegression(C=0.1, dual=False, fit_intercept=True, penalty="l2", solver="newton-cg") # BoW
# model = LogisticRegression(C=0.01, dual=True, fit_intercept=False, penalty="l2", solver="liblinear") # TF-IDF
model.fit(X=X_train, y=y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print(f"Training Accuracy = {classification_report(y_train, pred_train, output_dict=True)['accuracy']}")
print(f"Testing Accuracy = {classification_report(y_test, pred_test, output_dict=True)['accuracy']}")

Training Accuracy = 0.9759615384615384
Testing Accuracy = 0.5660377358490566


/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [27]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.54      0.96      0.69        26
           1       0.71      0.20      0.31        25
           2       0.00      0.00      0.00         2

    accuracy                           0.57        53
   macro avg       0.42      0.39      0.34        53
weighted avg       0.60      0.57      0.49        53



/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [28]:
# # Grid Search

# parameters = {
#     "penalty": ["l1", "l2", "elasticnet", "none"],
#     "dual": [True, False],
#     "C": [1, 0.1, 0.01],
#     "fit_intercept": [True, False],
#     "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
# }

# model = LogisticRegression()

# grid_search = GridSearchCV(model, parameters, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# grid_search.best_params_

#### Training Decision Tree Model

In [29]:
model = DecisionTreeClassifier(criterion="gini", max_features="sqrt", splitter="random") # BoW
# model = DecisionTreeClassifier(criterion="entropy", max_features="auto", splitter="random") # TF-IDF
model.fit(X=X_train, y=y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print(f"Training Accuracy = {classification_report(y_train, pred_train, output_dict=True)['accuracy']}")
print(f"Testing Accuracy = {classification_report(y_test, pred_test, output_dict=True)['accuracy']}")

Training Accuracy = 1.0
Testing Accuracy = 0.5283018867924528


In [30]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.56      0.96      0.70        26
           1       1.00      0.12      0.21        25
           2       0.00      0.00      0.00         2

    accuracy                           0.53        53
   macro avg       0.52      0.36      0.31        53
weighted avg       0.74      0.53      0.45        53



In [31]:
# # Grid Search

# parameters = {
#     "criterion": ["gini", "entropy", "log_loss"],
#     "splitter": ["best", "random"],
#     "max_features": ["auto", "sqrt", "log2", None],
# }

# model = DecisionTreeClassifier()

# grid_search = GridSearchCV(model, parameters, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# grid_search.best_params_

#### Training Random Forest Model

In [32]:
model = RandomForestClassifier() # BoW
# model = RandomForestClassifier(bootstrap=False, class_weight=None, 
#     criterion="entropy", max_features="sqrt", n_estimators=100, oob_score=False, warm_start=False) # TF-IDF
model.fit(X=X_train, y=y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print(f"Training Accuracy = {classification_report(y_train, pred_train, output_dict=True)['accuracy']}")
print(f"Testing Accuracy = {classification_report(y_test, pred_test, output_dict=True)['accuracy']}")

Training Accuracy = 1.0
Testing Accuracy = 0.5094339622641509


/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [33]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.51      0.96      0.67        26
           1       0.50      0.08      0.14        25
           2       0.00      0.00      0.00         2

    accuracy                           0.51        53
   macro avg       0.34      0.35      0.27        53
weighted avg       0.49      0.51      0.39        53



/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [34]:
# # Grid Search

# parameters = {
#     "n_estimators": [100, 200, 300],
#     "criterion": ["gini", "entropy", "log_loss"],
#     "max_features": ["auto", "sqrt", "log2", None],
#     "bootstrap": [True, False],
#     "oob_score": [True, False],
#     "warm_start": [True, False],
#     "class_weight": ["balanced", "balanced_subsample", None],
# }

# model = RandomForestClassifier()

# grid_search = GridSearchCV(model, parameters, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# grid_search.best_params_

#### Training Neural Network

In [35]:
# One Hot Encoding

encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
enc_y = enc_y.toarray()

print(f"Size of Labels = {enc_y.shape}")
print(f"Label Sample: {enc_y[0]}")

Size of Labels = (261, 3)
Label Sample: [0. 1. 0.]


In [36]:
# Train Test Split

X_train, X_test, y_train, y_test= train_test_split(X, enc_y, test_size=0.2, random_state=110)

print(f"Shape of Training Data: {X_train.shape}")
print(f"Shape of Training Labels: {y_train.shape}")
print(f"Shape of Testing Data: {X_test.shape}")
print(f"Shape of Testing Labels: {y_test.shape}")

Shape of Training Data: (208, 2001)
Shape of Training Labels: (208, 3)
Shape of Testing Data: (53, 2001)
Shape of Testing Labels: (53, 3)


In [37]:
# Define Custom Callback

class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold
 
    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

In [38]:
# Define Model
model = keras.models.Sequential([
    keras.layers.Dense(32, input_dim=X_train.shape[1], activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation="softmax"),
])

loss_function = keras.losses.CategoricalCrossentropy() # Define loss function
# loss_function = keras.losses.SparseCategoricalCrossentropy() # Define loss function
optimizer = keras.optimizers.SGD(learning_rate=0.005) # Define optimizer

callback = MyThresholdCallback(threshold=0.67) # Set callback at 67% accuracy

model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"]) # Compile the model

2022-09-19 14:11:18.478151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 14:11:18.478595: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-19 14:11:18.478678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-19 14:11:18.478752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-19 14:11:18.478826: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [39]:
# Train the model

history = model.fit(X_train, y_train, epochs=100, batch_size=1, 
    validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/100
208/208 [==============================] - 1s 3ms/step - loss: 0.9964 - accuracy: 0.5577 - val_loss: 0.9474 - val_accuracy: 0.4906
Epoch 2/100
208/208 [==============================] - 0s 2ms/step - loss: 0.9070 - accuracy: 0.5817 - val_loss: 0.9216 - val_accuracy: 0.4906
Epoch 3/100
208/208 [==============================] - 0s 2ms/step - loss: 0.8529 - accuracy: 0.6202 - val_loss: 0.8797 - val_accuracy: 0.4906
Epoch 4/100
208/208 [==============================] - 0s 2ms/step - loss: 0.8680 - accuracy: 0.6154 - val_loss: 0.8910 - val_accuracy: 0.4906
Epoch 5/100
208/208 [==============================] - 0s 2ms/step - loss: 0.8182 - accuracy: 0.6346 - val_loss: 0.8848 - val_accuracy: 0.4717
Epoch 6/100
208/208 [==============================] - 0s 2ms/step - loss: 0.7844 - accuracy: 0.6875 - val_loss: 0.8946 - val_accuracy: 0.4906
Epoch 7/100
208/208 [==============================] - 0s 2ms/step - loss: 0.7886 - accuracy: 0.6971 - val_loss: 0.8904 - val_accuracy: 0.5283

In [40]:
pred_train = [np.argmax(ele) for ele in model.predict(X_train)]
pred_test = [np.argmax(ele) for ele in model.predict(X_test)]

new_y_train = [np.argmax(ele) for ele in y_train]
new_y_test = [np.argmax(ele) for ele in y_test]

print(f"Training Accuracy = {classification_report(new_y_train, pred_train, output_dict=True)['accuracy']}")
print(f"Testing Accuracy = {classification_report(new_y_test, pred_test, output_dict=True)['accuracy']}")

2/2 [==============================] - 0s 3ms/step
Training Accuracy = 0.9903846153846154
Testing Accuracy = 0.6981132075471698


/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [41]:
print(classification_report(new_y_test, pred_test))

              precision    recall  f1-score   support

           0       0.66      0.88      0.75        26
           1       0.78      0.56      0.65        25
           2       0.00      0.00      0.00         2

    accuracy                           0.70        53
   macro avg       0.48      0.48      0.47        53
weighted avg       0.69      0.70      0.68        53



/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta